In [5]:
# todo: 월 주가 데이터 조회 및 캐싱
from base import YearMonth
from repository.maria import chart


begin = YearMonth(2000, 1)
end = YearMonth(2022, 10)

df = chart.month_chart(begin, end)
df["date"]


0         2000-01-31
1         2000-01-31
2         2000-01-31
3         2000-01-31
4         2000-01-31
             ...    
453762    2022-10-31
453763    2022-10-31
453764    2022-10-31
453765    2022-10-31
453766    2022-10-31
Name: date, Length: 453767, dtype: object